## Sistemas de Recomendación

***Lo primero que tenemos que hacer para hacer un Sistema de Recomendación es instalar la libreria Surprise.***

In [ ]:
!pip install scikit-surprise

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 772.0/772.0 kB 18.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.3-cp310-cp310-linux_x86_64.whl size=3096325 sha256=e6b3522e7f1e4bad1e2781e4900d3d119373523941404b0d84a91f3751d7de47
  Stored in directory: /root/.cache/pip/wheels/a5/ca/a8/4e28def53797fdc4363ca4af740db15a9c2f1595ebc51fb445
Successfully built scikit-surprise


***También tenemos que importar las librerías que nos harán falta.***

In [ ]:
import pandas as pd
from surprise import KNNBasic, KNNWithMeans
from surprise import Dataset
from surprise import Reader
from surprise import accuracy
from surprise.model_selection import train_test_split

***Con la Class Reader hacemos la ingesta de datos.***

***Con line_format especificamos el formato de cada línea. Tendrá identificador de usuario, identificador de película, calificación dada por el usuario y la marca del tiempo.***

***Y el separador en cada línea son ::***

In [ ]:
reader = Reader(line_format = "user item rating timestamp", sep = '::')
data = Dataset.load_from_file('/content/ratings.dat', reader = reader)

In [ ]:
type(data)

surprise.dataset.DatasetAutoFolds

***Para ver las primeras 5 líneas.***

In [ ]:
lineas = data.raw_ratings
for i in range(5):
  print(lineas[i])

('1', '1193', 5.0, '978300760')
('1', '661', 3.0, '978302109')
('1', '914', 3.0, '978301968')
('1', '3408', 4.0, '978300275')
('1', '2355', 5.0, '978824291')


***Nos aseguramos de los rangos mínimos y máximos del reiting.***

In [ ]:
min_rating, max_rating = reader.rating_scale
print(f"El rango minimo es:",min_rating, "y el maximo es:", max_rating)

El rango minimo es: 1 y el maximo es: 5


***Dividimos los datos en train y test, dejando para el test el 30% de los datos.***

In [ ]:
train, test = train_test_split(data, test_size = 0.3)

***Nos basaremos en los algoritmos de vecino más cercano KNNBasic y KNNWithMeans.***

***Para el KNN el parámetro k indica el número de vecinos más cercanos a considerar al realizar las predicciones. El parámetro "sim_options" se utiliza para configurar las opciones de similitud entre elementos. En este caso, se utiliza la similitud de Pearson para calcular la similitud entre los usuarios. El atributo "user_name" se establece en True, lo cual sugiere que se están considerando los nombres de los usuarios en el cálculo de similitud.***

***En el modelo KNNWithMeans también se utiliza el algoritmo de vecinos más cercanos, pero con la diferencia de que tiene en cuenta las calificaciones medias de los usuarios (means). El parámetro "sim_options" se utiliza para configurar las opciones de similitud entre elementos. En este caso, se utiliza la similitud del coseno para calcular la similitud entre los elementos, y el atributo "user_based" se establece en False, lo cual sugiere que se están considerando las similitudes entre los elementos (por ejemplo, películas) en lugar de entre los usuarios.***

In [ ]:
knn = KNNBasic(k = 50, sim_options = {'name':'pearson', 'user_name': True})
kMeans = KNNWithMeans(k = 50, sim_options= {'name':'cosine', 'user_based': False})

***Vamos a entrenar los modelos.***

In [ ]:
knn.fit(train)
kMeans.fit(train)

Computing the pearson similarity matrix...
Done computing similarity matrix.
Computing the cosine similarity matrix...
Done computing similarity matrix.


***Vamos a probarlo a mano, para el usuario 42 que cualificación daría para la película 7.***

In [ ]:
user_id = 42
item_id = 7
knn_user_prediction = knn.predict(user_id, item_id)
kMeans_user_prediction = kMeans.predict(user_id, item_id)
knn_user_prediction
kMeans_user_prediction

Prediction(uid=42, iid=7, r_ui=None, est=3.5826213389778703, details={'was_impossible': True, 'reason': 'User and/or item is unknown.'})

***Vemos que el usuario 42 (uid=42) no ha calificado (r_ui=None) la película 7 (iid=7). La calificación estimada sería de 3.583.***

***Calculamos la precision de la predicción. Con KNN acertamos más.***

In [ ]:
knn_test_prediction = knn.test(test)
kMeans_test_prediction = kMeans.test(test)
knn_rmse = accuracy.rmse(knn_test_prediction)
kMeans_rmse = accuracy.rmse(kMeans_test_prediction)

RMSE: 0.9658
RMSE: 0.8985


In [ ]:
print(f"KNN: ", knn_rmse )
print(f"kMeans: ", kMeans_rmse)

KNN:  0.9658255413310246
kMeans:  0.8984737027699121
